In [2]:
import numpy as np

np.set_printoptions(suppress=True)
def FK_PoE(q, a, rot, jt, M):

    T = POE(q, a, rot, jt)
    Tf = np.dot(T, M)

    R = Tf[:3, :3]
    p = Tf[:3, 3]
    return R, p

def POE(q, a, rot, jt):

    T = np.eye(4)
    n = len(q)

    for ii in range(n - 1, -1, -1):
        if jt[ii] == 'R':
            rot_hat = np.array([
                [0, -rot[ii, 2], rot[ii, 1]],
                [rot[ii, 2], 0, -rot[ii, 0]],
                [-rot[ii, 1], rot[ii, 0], 0]
            ])
            e_rot_hat = np.eye(3) + rot_hat * np.sin(q[ii]) + np.dot(rot_hat, rot_hat) * (1 - np.cos(q[ii]))
        elif jt[ii] == 'P':
            rot_hat = np.zeros((3, 3))
            e_rot_hat = np.eye(3)

        if jt[ii] == 'R' and ii > 0:
            Sv = -np.cross(rot[ii, :], a[ii, :])
        elif jt[ii] == 'R' and ii == 0:
            Sv = np.array([0, 0, 0])
        elif jt[ii] == 'P':
            Sv = a[ii, :]

        p = (np.eye(3) * q[ii] + (1 - np.cos(q[ii])) * rot_hat +
             (q[ii] - np.sin(q[ii])) * np.dot(rot_hat, rot_hat)).dot(Sv)

        e_zai = np.block([
            [e_rot_hat, p.reshape(3, 1)],
            [np.zeros((1, 3)), 1]
        ])

        T = np.dot(e_zai, T)

    return T

# Example inputs
t1 = 0
t2 = -np.pi / 2
t3 = 0
t4 = 0
t5 = np.pi / 2
t6 = 0
theta = np.array([t1, t2, t3, t4, t5, t6])

W1 = 109 / 1000
W2 = 82 / 1000
L1 = 425 / 1000
L2 = 392 / 1000
H1 = 89 / 1000
H2 = 95 / 1000

a = np.array([[0, 0, 0], [0, 0, H1], [-L1, 0, H1], [-L1 - L2, 0, H1],
              [-L1 - L2, -W1, 0], [-L1 - L2, 0, H1 - H2]])
rot = np.array([[0, 0, 1], [0, -1, 0], [0, -1, 0], [0, -1, 0], [0, 0, -1], [0, -1, 0]])
jt = 'RRRRRR'
M = np.array([[1, 0, 0, -L1 - L2],
              [0, 0, -1, -W1 - W2],
              [0, 1, 0, H1 - H2],
              [0, 0, 0, 1]])

# Calculate forward kinematics
R, p = FK_PoE(theta, a, rot, jt, M)


print("Rotation Matrix (R):\n", R)
print("Position Vector (p):\n", p)


Rotation Matrix (R):
 [[ 0.  1. -0.]
 [-1.  0. -0.]
 [-0.  0.  1.]]
Position Vector (p):
 [-0.095 -0.109  0.988]
